Data Extraction

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
# CONFIG
DATASET_SLUG = "rohanpanda80/us-sec-financial-statement-2021-2024"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 30)

In [3]:
def get_sec_root(dest_path="kdata"):
    """
    Returns the path to the root SEC dataset folder containing the quarter subfolders.
    If already downloaded, uses existing files. Otherwise downloads from Kaggle.
    Works for local, Colab, and Kaggle environments.
    """
    dest_path = Path(dest_path)
    dest_path.mkdir(exist_ok=True)

    # Look for existing folders with quarter subfolders
    existing_dirs = [d for d in dest_path.iterdir() if d.is_dir()]
    for d in existing_dirs:
        if any("q" in q.name for q in d.iterdir() if q.is_dir()):
            print(f"Using existing dataset at {d}")
            return d

    # If not found, download from Kaggle
    print("Dataset not found locally. Downloading from Kaggle...")
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(DATASET_SLUG, path=dest_path, unzip=True)

    # Handle nested folder (Kaggle often wraps data in a subfolder)
    extracted_dirs = [d for d in dest_path.iterdir() if d.is_dir()]
    for d in extracted_dirs:
        if any("q" in q.name for q in d.iterdir() if q.is_dir()):
            print(f"Dataset ready at {d}")
            return d

    # Fallback: just return dest_path if structure is unusual
    print(f"Dataset ready at {dest_path}")
    return dest_path

In [4]:
def get_quarter_dirs(sec_root):
    """
    Returns a sorted list of quarter directories under the SEC root.
    """
    return sorted([d for d in sec_root.iterdir() if d.is_dir() and "q" in d.name])

In [5]:
# LOAD ALL SUB & NUM FILES
def load_all_data():
    """
    Load all sub.txt and num.txt files across all quarters and return merged dataframes.
    """
    sec_root = get_sec_root()
    quarter_dirs = get_quarter_dirs(sec_root)

    if not quarter_dirs:
        raise FileNotFoundError(f"No quarter directories found in {sec_root}")

    all_sub, all_num = [], []

    for qdir in quarter_dirs:
        sub_path = qdir / "sub.txt"
        num_path = qdir / "num.txt"

        if sub_path.exists() and num_path.exists():
            print(f"Loading {qdir.name} ...")
            df_sub = pd.read_csv(sub_path, sep="\t", low_memory=False)
            df_num = pd.read_csv(num_path, sep="\t", low_memory=False)

            # Add quarter info
            df_sub["quarter"] = qdir.name
            df_num["quarter"] = qdir.name

            all_sub.append(df_sub)
            all_num.append(df_num)
        else:
            print(f"Skipping {qdir.name}, files not found")

    # Combine all quarters
    sub_full = pd.concat(all_sub, ignore_index=True)
    num_full = pd.concat(all_num, ignore_index=True)

    return sub_full, num_full

In [6]:
def merge_dataset(sub_df, num_df, key, how):
    return pd.merge(sub_df, num_df, on=key, how=how)

In [7]:
sub_df, num_df = load_all_data()

Using existing dataset at kdata\Dataset
Loading 2022q1 ...
Loading 2022q2 ...
Loading 2022q2 ...
Loading 2022q3 ...
Loading 2022q3 ...
Loading 2022q4 ...
Loading 2022q4 ...
Loading 2023q1 ...
Loading 2023q1 ...
Loading 2023q2 ...
Loading 2023q2 ...
Loading 2023q3 ...
Loading 2023q3 ...
Loading 2023q4 ...
Loading 2023q4 ...
Loading 2024q1 ...
Loading 2024q1 ...
Loading 2024q2 ...
Loading 2024q2 ...
Loading 2024q3 ...
Loading 2024q3 ...
Loading 2024q4 ...
Loading 2024q4 ...


In [8]:
sub_df.head()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,quarter
0,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1
1,0000002488-22-000016,2488,ADVANCED MICRO DEVICES INC,3674.0,US,CA,SANTA CLARA,95054,2485 AUGUSTINE DRIVE,NaN,(408) 749-4000,US,CA,SANTA CLARA,95054,2485 AUGUSTINE DRIVE,NaN,US,DE,941692300.0,NaN,NaN,1-LAF,1,1231.0,10-K,20211231.0,2021.0,FY,20220203,2022-02-03 17:23:00.0,0,1,amd-20211225_htm.xml,1,NaN,2022q1
2,0000002969-22-000010,2969,AIR PRODUCTS & CHEMICALS INC /DE/,2810.0,US,PA,ALLENTOWN,18106-5500,1940 AIR PRODUCTS BLVD.,NaN,6104814911,US,PA,ALLENTOWN,18106-5500,1940 AIR PRODUCTS BLVD.,NaN,US,DE,231274455.0,NaN,NaN,1-LAF,0,930.0,10-Q,20211231.0,2022.0,Q1,20220204,2022-02-04 10:32:00.0,0,1,apd-20211231_htm.xml,1,NaN,2022q1
3,0000003499-22-000004,3499,ALEXANDERS INC,6798.0,US,NJ,PARAMUS,07652,210 ROUTE 4 EAST,NaN,201-587-8541,US,NJ,PARAMUS,07652,210 ROUTE 4 EAST,NaN,US,DE,510100517.0,NaN,NaN,2-ACC,0,1231.0,10-K,20211231.0,2021.0,FY,20220214,2022-02-14 08:19:00.0,0,1,alx-20211231_htm.xml,1,NaN,2022q1
4,0000003570-22-000024,3570,CHENIERE ENERGY INC,4924.0,US,TX,HOUSTON,77002,700 MILAM ST.,SUITE 1900,7133755000,US,TX,HOUSTON,77002,700 MILAM ST.,SUITE 1900,US,DE,954352386.0,CHENIERE ENERGY INC,19960827.0,1-LAF,1,1231.0,10-K,20211231.0,2021.0,FY,20220224,2022-02-23 21:37:00.0,0,1,lng-20211231_htm.xml,1,NaN,2022q1


In [9]:
num_df.head()

,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote,quarter
0,0001126975-22-000070,OtherComprehensiveIncomeLossNetOfTax,us-gaap/2021,20201231,4,USD,PartnerCapitalComponents=AccumulatedOtherCompr...,NaN,-21100000.0,NaN,2022q1
1,0001748824-22-000018,AdjustmentsRelatedToTaxWithholdingForShareBase...,us-gaap/2021,20211231,4,USD,ConsolidatedEntities=ConsolidatedEntityExcludi...,NaN,31300000.0,NaN,2022q1
2,0000012927-22-000010,Revenues,us-gaap/2021,20211231,4,USD,BusinessSegments=GlobalServices;ConsolidationI...,NaN,248000000.0,NaN,2022q1
3,0001564590-22-006237,OtherAssetsNoncurrent,us-gaap/2021,20201231,0,USD,BusinessSegments=TruckPartsAndOther;,NaN,998900000.0,NaN,2022q1
4,0000107815-22-000116,DefinedBenefitPlanAssetsForPlanBenefitsNoncurrent,us-gaap/2021,20211231,0,USD,RetirementPlanType=PensionPlansDefinedBenefit;,NaN,389000000.0,NaN,2022q1


In [10]:
df = merge_dataset(sub_df, num_df, 'adsh', 'inner')

In [11]:
df.shape

(41260371, 47)

In [12]:
def dataframe_summary(df):
    summary = pd.DataFrame({
        "Column": df.columns,
        "dtype": df.dtypes,
        "num_rows": len(df),
        "num_unique": df.nunique(),
        "num_missing": df.isna().sum(),
        "pct_missing": ((df.isna().sum() / len(df)) * 100).round(2).astype(float)
    })
    summary.reset_index(drop=True, inplace=True)
    return summary

<h1>Filtering only tags matching with rev_patterns</h1>

In [13]:
# Revenue patterns
rev_patterns = ['revenue', 'revenues']

In [14]:
def cleaned_revenue(df, rev_patterns):
    # Getting all the partial revenue match
    df_rev = df[df['tag'].str.lower().isin(rev_patterns)]
    df_clean = df_rev.reset_index(drop=True)
    return df_clean

In [15]:
# Filtering revenues only
df_rev = cleaned_revenue(df, rev_patterns)
df_rev.head()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,quarter_x,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote,quarter_y
0,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20211231,4,USD,BusinessSegments=CrudeOilMarketing;Consolidati...,NaN,1930042000.0,NaN,2022q1
1,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20211231,4,USD,ConsolidationItems=CorporateNonSegment;,NaN,0.0,NaN,2022q1
2,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20211231,4,USD,ProductOrService=PipelineAndStorage;,NaN,664000.0,NaN,2022q1
3,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20201231,4,USD,BusinessSegments=Transportation;ConsolidationI...,NaN,71724000.0,NaN,2022q1
4,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,17 S. BRIAR HOLLOW LN.,NaN,713-881-3600,US,TX,HOUSTON,77001,P O BOX 844,NaN,US,DE,741753147.0,ADAMS RESOURCES & ENERGY INC,19920703.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220309,2022-03-09 16:17:00.0,0,1,ae-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20210331,1,USD,NaN,NaN,325491000.0,NaN,2022q1


In [16]:
# Checking for NaN countries, city, zip and state in Business Address
df_rev[df_rev["countryba"].isna()].head()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,quarter_x,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote,quarter_y
29388,0001254699-22-000010,1254699,QVC INC,5961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,PA,WEST CHESTER,19380,1200 WILSON DRIVE AT STUDIO PARK,NaN,US,DE,232414041.0,NaN,NaN,4-NON,1,1231.0,10-K,20211231.0,2021.0,FY,20220225,2022-02-25 16:53:00.0,0,1,qvc-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20201231,4,USD,NaN,NaN,11472000000.0,NaN,2022q1
29389,0001254699-22-000010,1254699,QVC INC,5961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,PA,WEST CHESTER,19380,1200 WILSON DRIVE AT STUDIO PARK,NaN,US,DE,232414041.0,NaN,NaN,4-NON,1,1231.0,10-K,20211231.0,2021.0,FY,20220225,2022-02-25 16:53:00.0,0,1,qvc-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20191231,4,USD,NaN,NaN,10986000000.0,NaN,2022q1
29390,0001254699-22-000010,1254699,QVC INC,5961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,PA,WEST CHESTER,19380,1200 WILSON DRIVE AT STUDIO PARK,NaN,US,DE,232414041.0,NaN,NaN,4-NON,1,1231.0,10-K,20211231.0,2021.0,FY,20220225,2022-02-25 16:53:00.0,0,1,qvc-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20211231,4,USD,NaN,NaN,11354000000.0,NaN,2022q1
42764,0001600626-22-000021,1600626,"GRIFFIN REALTY TRUST, INC.",6798.0,NaN,NaN,NaN,NaN,NaN,NaN,3104696100,US,CA,EL SEGUNDO,90245,1520 EAST GRAND AVENUE,NaN,NaN,NaN,464654479.0,"GRIFFIN CAPITAL ESSENTIAL ASSET REIT, INC.",20190710.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220228,2022-02-28 17:32:00.0,0,1,gcnl-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20211231,4,USD,NaN,NaN,459872000.0,NaN,2022q1
42765,0001600626-22-000021,1600626,"GRIFFIN REALTY TRUST, INC.",6798.0,NaN,NaN,NaN,NaN,NaN,NaN,3104696100,US,CA,EL SEGUNDO,90245,1520 EAST GRAND AVENUE,NaN,NaN,NaN,464654479.0,"GRIFFIN CAPITAL ESSENTIAL ASSET REIT, INC.",20190710.0,4-NON,0,1231.0,10-K,20211231.0,2021.0,FY,20220228,2022-02-28 17:32:00.0,0,1,gcnl-20211231_htm.xml,1,NaN,2022q1,Revenues,us-gaap/2021,20191231,4,USD,NaN,NaN,387108000.0,NaN,2022q1


In [17]:
# Filling out the NaN values with values from Mailing Address
fields = ["country", "stpr", "city", "zip"]

for f in fields:
    df_rev[f + "ba"] = df_rev[f + "ba"].fillna(df_rev[f + "ma"])

<h1>Renaming & Dropping Columns</h1>

In [18]:
# Excluding unnecessary columns
df_final = df_rev.copy()
df_final = df_final.drop(columns=['countryma', 'stprma', 'cityma', 
                              'zipma', 'bas1', 'bas2','mas1', 'mas2', 'ein',
                              'former', 'changed', 'footnote', 'wksi',
                              'afs', 'accepted', 'prevrpt', 'fye', 'fp', 
                              'quarter_x', 'detail', 'instance', 'version', 'tag',
                              'countryinc', 'stprinc', 'coreg', 'segments', 'form'])

In [19]:
df_final.head()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,baph,period,fy,filed,nciks,aciks,ddate,qtrs,uom,value,quarter_y
0,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021.0,20220309,1,NaN,20211231,4,USD,1930042000.0,2022q1
1,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021.0,20220309,1,NaN,20211231,4,USD,0.0,2022q1
2,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021.0,20220309,1,NaN,20211231,4,USD,664000.0,2022q1
3,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021.0,20220309,1,NaN,20201231,4,USD,71724000.0,2022q1
4,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021.0,20220309,1,NaN,20210331,1,USD,325491000.0,2022q1


<h1>Mapping the columns as per the requirement<br></h1>

In [20]:
# Renaming the column names
df_final = df_final.rename(columns={
    "adsh": "accession_number",
    "name": "company",
    "countryba": "country",
    "stprba": "state",
    "cityba": "city",
    "zipba": "zip",
    "baph": "phone_no",
    "fy": "fiscal_year",
    "value": "revenue",
    "uom": "revenue_unit",
    "quarter_y": "file_name"}
)

In [21]:
# Filling the missing fiscal year from year column
df_final["fiscal_year"] = df_final["fiscal_year"].fillna(
    df_final["period"]
    .astype(str)
    .str.split(".")
    .str[0]
    .str[:4]
)

In [22]:
df_final['fiscal_year'].unique()

array([2021.0, 2022.0, 2020.0, '2021', 2019.0, '2022', '2020', 2023.0,
       2015.0, 2016.0, 2017.0, 2018.0, '2023', 2024.0, 2012.0, 2013.0,
       2025.0, '2024'], dtype=object)

In [23]:
# Changing data type
df_final["accession_number"] = df_final["accession_number"].astype("string")
df_final["company"] = df_final["company"].astype("string")
df_final["country"] = df_final["country"].astype("string")
df_final["state"] = df_final["state"].astype("string")
df_final["city"] = df_final["city"].astype("string")
df_final["zip"] = df_final["zip"].astype("string")
df_final["phone_no"] = df_final["phone_no"].astype("string")
df_final["revenue_unit"] = df_final["revenue_unit"].astype("string")
df_final["file_name"] = df_final["file_name"].astype("string")
df_final['aciks'] = df_final['aciks'].astype("string")
df_final['fiscal_year'] = df_final['fiscal_year'].astype(int)

# Filling in null values
df_final['aciks'] = df_final['aciks'].fillna('')
df_final["phone_no"] = df_final["phone_no"].fillna('')
df_final["phone_no"] = df_final["phone_no"].fillna('')
df_final["state"] = df_final["state"].fillna('')
df_final["city"] = df_final["city"].fillna('')
df_final["zip"] = df_final["zip"].fillna('')
df_final["revenue"] = df_final["revenue"].fillna(0)

In [24]:
dataframe_summary(df_final)

,Column,dtype,num_rows,num_unique,num_missing,pct_missing
0,accession_number,string[python],619300,29935,0,0.000000000000000000000000000000
1,cik,int64,619300,3654,0,0.000000000000000000000000000000
2,company,string[python],619300,3914,0,0.000000000000000000000000000000
3,sic,float64,619300,357,315,0.050000000000000002775557561563
4,country,string[python],619300,70,8,0.000000000000000000000000000000
5,state,string[python],619300,60,0,0.000000000000000000000000000000
6,city,string[python],619300,1448,0,0.000000000000000000000000000000
7,zip,string[python],619300,2471,0,0.000000000000000000000000000000
8,phone_no,string[python],619300,3889,0,0.000000000000000000000000000000
9,period,float64,619300,94,0,0.000000000000000000000000000000


Mapping sic to industry

In [25]:
PROJECT_ROOT = Path(".").resolve()
DATA_DIR = PROJECT_ROOT / "data"
sic_map_path = DATA_DIR / "sic_mapping.csv"

if not sic_map_path.exists():
    raise FileNotFoundError(f"SIC mapping file not found at {sic_map_path}")

# Loading the sic_mapping file
sic_map = pd.read_csv(sic_map_path, encoding="latin1")

sic_dict = dict(zip(sic_map["SIC Code"], sic_map["Industry Title"]))

In [26]:
# Mapping sic values to their industries
df_final["industry"] = df_final["sic"].map(sic_dict).fillna("Unknown Industry")
df_final.head()

,accession_number,cik,company,sic,country,state,city,zip,phone_no,period,fiscal_year,filed,nciks,aciks,ddate,qtrs,revenue_unit,revenue,file_name,industry
0,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021,20220309,1,,20211231,4,USD,1930042000.0,2022q1,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
1,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021,20220309,1,,20211231,4,USD,0.0,2022q1,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
2,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021,20220309,1,,20211231,4,USD,664000.0,2022q1,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
3,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021,20220309,1,,20201231,4,USD,71724000.0,2022q1,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
4,0000002178-22-000033,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,US,TX,HOUSTON,77027,713-881-3600,20211231.0,2021,20220309,1,,20210331,1,USD,325491000.0,2022q1,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...


<h1>KPI</h1>
<p>
This section computes key revenue KPIs used in analysis.
</p>
<ul>
        <li><strong>`cik`</strong> — Central Index Key, unique company identifier used for grouping.</li>
        <li><strong>`fiscal_year`</strong> — integer year (e.g. `2023`).</li>
        <li><strong>`qtrs`</strong> — quarter number 1..4 (integer). Use this to identify quarter within `fiscal_year`.</li>
        <li><strong>`revenue`</strong> — numeric revenue value.</li>
        <li><strong>`revenue_unit`</strong> — currency unit (e.g. `USD`, `INR`).</li>
</ul>
<p>
KPIs computed:
</p>
<ul>
    <li><strong>Revenue growth</strong> — period-over-period percent change grouped by `cik` (adds `revenue_growth_pct`).</li>
    <li><strong>CAGR</strong> — compound annual growth rate per `cik` calculated from first to last available observation; returned as percent in column `cagr` (e.g. `12.34` = 12.34%).</li>
    <li><strong>Quarterly revenue</strong> — aggregated revenue by `cik`, `fiscal_year` and `qtrs` (returns a tidy DataFrame with `qtr_num` and `quarter_label`).</li>
</ul>
<p>
These functions assume `fiscal_year` and `qtrs` exist and are correctly typed. If `revenue_unit` varies across rows, consider normalizing units before KPI calculations.
</p>

In [34]:
# Revenue Growth (%)
def compute_revenue_growth(df):
    """
    Compute period-over-period revenue growth (%) per `cik` using `fiscal_year` and `qtrs`.

    - Expects `fiscal_year` (int) and `qtrs` (1-4) to be present.
    - Sorts by `cik`, `fiscal_year`, `qtr_num` and computes percentage change.
    Returns a copy of the dataframe with `revenue_growth_pct` column added.
    """
    df = df.copy()

    df['qtr_num'] = pd.to_numeric(df['qtrs'], errors='coerce').fillna(0).astype(int)
    df = df.sort_values(['cik', 'fiscal_year', 'qtr_num'])
    df['revenue_growth_pct'] = df.groupby('cik')['revenue'].pct_change() * 100
    # Replace infinite changes (e.g., 0 -> positive) with NaN and round
    df['revenue_growth_pct'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df['revenue_growth_pct'] = df['revenue_growth_pct'].round(4)
    return df

In [35]:
# CAGR (uses first and last available observation per cik)
def compute_cagr(df):
    """
    Compute CAGR per `cik` based on first and last available `revenue`.
    Uses `fiscal_year` (falls back to `year` or `period`). Returns dataframe
    merged with a `cagr` column (decimal, e.g. 0.12 = 12%).
    """

    # Ensure quarter numeric column exists
    if 'qtrs' in df.columns:
        df['qtr_num'] = pd.to_numeric(df['qtrs'], errors='coerce').fillna(0).astype(int)
    else:
        df['qtr_num'] = 0

    def cagr_for_group(g):
        g = g.sort_values(['fiscal_year', 'qtr_num'])
        if len(g) < 2:
            return np.nan
        first_val = g.iloc[0]['revenue']
        last_val = g.iloc[-1]['revenue']
        first_year = int(g.iloc[0]['fiscal_year'])
        last_year = int(g.iloc[-1]['fiscal_year'])
        first_q = int(g.iloc[0]['qtr_num'])
        last_q = int(g.iloc[-1]['qtr_num'])
        # compute time difference in years including quarter offsets
        years = (last_year - first_year) + (last_q - first_q) / 4.0
        if years <= 0 or first_val == 0 or pd.isna(first_val) or pd.isna(last_val):
            return np.nan
        # Return CAGR as percentage (e.g. 12.34 = 12.34%)
        try:
            cagr_percent = ((last_val / first_val) ** (1.0 / years) - 1) * 100
        except Exception:
            return np.nan
        return round(cagr_percent, 4)

    cagr_series = df.groupby('cik').apply(cagr_for_group)
    cagr_series.name = 'cagr'
    df = df.merge(cagr_series, left_on='cik', right_index=True, how='left')
    return df

In [36]:
# Revenue per Quarter
def compute_quarter_revenue(df):
    """
    Aggregate revenue per `cik` by fiscal year and quarter.
    Returns a dataframe with `cik`, `fiscal_year`, `quarter_num`, `revenue` and `quarter_label`.
    """
    df = df.copy()

    if 'qtrs' in df.columns:
        df['qtr_num'] = pd.to_numeric(df['qtrs'], errors='coerce').fillna(0).astype(int)
    else:
        df['qtr_num'] = 0

    quarter_rev = df.groupby(['cik', 'fiscal_year', 'qtr_num'], as_index=False)['revenue'].sum()
    quarter_rev['quarter_label'] = quarter_rev['fiscal_year'].astype(str) + 'Q' + quarter_rev['qtr_num'].astype(str)
    return quarter_rev

In [34]:
# Calling all the kpi function
df_kpi = df_final.copy()

In [ ]:
df_kpi_rev_growth = compute_revenue_growth(df_kpi)
df_kpi_rev_growth

In [ ]:
df_kpi_cagr = compute_cagr(df_kpi)
df_kpi_cagr

In [ ]:
df_kpi_quar_rev = compute_quarter_revenue(df_kpi)
df_kpi_quar_rev

<h1>Test Dataset for KPI</h1>

In [37]:
# Small test dataset to exercise KPI functions
data = [
    # cik 1000: three quarters (steady growth)
    {
        "accession_number": "A-1000-1",
        "cik": 1000,
        "company": "Test Co A",
        "sic": 1234.0,
        "country": "US",
        "state": "CA",
        "city": "SF",
        "zip": "94105",
        "phone_no": "000-000-0000",
        "period": "20210331.0",
        "fiscal_year": 2021,
        "filed": "20210401",
        "nciks": "",
        "aciks": "",
        "ddate": "20210331",
        "qtrs": 1,
        "revenue_unit": "USD",
        "revenue": 100.0,
        "file_name": "2021q1",
        "industry": "Test Industry"
    },
    {
        "accession_number": "A-1000-2",
        "cik": 1000,
        "company": "Test Co A",
        "sic": 1234.0,
        "country": "US",
        "state": "CA",
        "city": "SF",
        "zip": "94105",
        "phone_no": "000-000-0000",
        "period": "20210630.0",
        "fiscal_year": 2021,
        "filed": "20210701",
        "nciks": "",
        "aciks": "",
        "ddate": "20210630",
        "qtrs": 2,
        "revenue_unit": "USD",
        "revenue": 120.0,
        "file_name": "2021q2",
        "industry": "Test Industry"
    },
    {
        "accession_number": "A-1000-3",
        "cik": 1000,
        "company": "Test Co A",
        "sic": 1234.0,
        "country": "US",
        "state": "CA",
        "city": "SF",
        "zip": "94105",
        "phone_no": "000-000-0000",
        "period": "20210930.0",
        "fiscal_year": 2021,
        "filed": "20211001",
        "nciks": "",
        "aciks": "",
        "ddate": "20210930",
        "qtrs": 3,
        "revenue_unit": "USD",
        "revenue": 150.0,
        "file_name": "2021q3",
        "industry": "Test Industry"
    },

    # cik 2000: single observation (CAGR should be NaN)
    {
        "accession_number": "B-2000-1",
        "cik": 2000,
        "company": "Solo Corp",
        "sic": 4321.0,
        "country": "US",
        "state": "NY",
        "city": "NYC",
        "zip": "10001",
        "phone_no": "111-111-1111",
        "period": "20211231.0",
        "fiscal_year": 2021,
        "filed": "20220115",
        "nciks": "",
        "aciks": "",
        "ddate": "20211231",
        "qtrs": 4,
        "revenue_unit": "USD",
        "revenue": 500.0,
        "file_name": "2021q4",
        "industry": "Solo Industry"
    },

    # cik 3000: zero revenue then recovery (tests zero handling)
    {
        "accession_number": "C-3000-1",
        "cik": 3000,
        "company": "Recover Inc",
        "sic": 5555.0,
        "country": "US",
        "state": "TX",
        "city": "Houston",
        "zip": "77002",
        "phone_no": "222-222-2222",
        "period": "20210630.0",
        "fiscal_year": 2021,
        "filed": "20210710",
        "nciks": "",
        "aciks": "",
        "ddate": "20210630",
        "qtrs": 2,
        "revenue_unit": "USD",
        "revenue": 0.0,
        "file_name": "2021q2",
        "industry": "Recovery Industry"
    },
    {
        "accession_number": "C-3000-2",
        "cik": 3000,
        "company": "Recover Inc",
        "sic": 5555.0,
        "country": "US",
        "state": "TX",
        "city": "Houston",
        "zip": "77002",
        "phone_no": "222-222-2222",
        "period": "20210930.0",
        "fiscal_year": 2021,
        "filed": "20211010",
        "nciks": "",
        "aciks": "",
        "ddate": "20210930",
        "qtrs": 3,
        "revenue_unit": "USD",
        "revenue": 200.0,
        "file_name": "2021q3",
        "industry": "Recovery Industry"
    },
]

df_test = pd.DataFrame(data)

# Quick display
print(df_test[["cik", "fiscal_year", "qtrs", "revenue"]])

    cik  fiscal_year  qtrs  revenue
0  1000         2021     1    100.0
1  1000         2021     2    120.0
2  1000         2021     3    150.0
3  2000         2021     4    500.0
4  3000         2021     2      0.0
5  3000         2021     3    200.0


In [43]:
# Now run the KPI functions (assumes functions are defined in the notebook)
df_g = compute_revenue_growth(df_test)
print('Revenue growth (sample):')
print(df_g[['cik','fiscal_year','qtrs','revenue','revenue_growth_pct']])

Revenue growth (sample):
    cik  fiscal_year  qtrs  revenue  revenue_growth_pct
0  1000         2021     1    100.0                 NaN
1  1000         2021     2    120.0                20.0
2  1000         2021     3    150.0                25.0
3  2000         2021     4    500.0                 NaN
4  3000         2021     2      0.0                 NaN
5  3000         2021     3    200.0                 NaN


C:\Users\sw\AppData\Local\Temp\ipykernel_25784\383294432.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['revenue_growth_pct'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [42]:

df_c = compute_cagr(df_test)
print('CAGR results (added to rows):')
print(df_c[['cik','fiscal_year','qtrs','revenue','cagr']].drop_duplicates(subset=['cik']) )


CAGR results (added to rows):
    cik  fiscal_year  qtrs  revenue   cagr
0  1000         2021     1    100.0  125.0
3  2000         2021     4    500.0    NaN
4  3000         2021     2      0.0    NaN


C:\Users\sw\AppData\Local\Temp\ipykernel_25784\1602838224.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cagr_series = df.groupby('cik').apply(cagr_for_group)


In [41]:
df_q = compute_quarter_revenue(df_test)
print('Quarterly aggregated revenue:')
print(df_q)

Quarterly aggregated revenue:
    cik  fiscal_year  qtr_num  revenue quarter_label
0  1000         2021        1    100.0        2021Q1
1  1000         2021        2    120.0        2021Q2
2  1000         2021        3    150.0        2021Q3
3  2000         2021        4    500.0        2021Q4
4  3000         2021        2      0.0        2021Q2
5  3000         2021        3    200.0        2021Q3
